# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,252.48,59,20,4.00,RU,['dt']
1,1,mejit,10.2753,170.8646,300.20,80,95,10.34,MH,['dt']
2,2,blackmans bay,-43.0167,147.3167,289.55,60,18,1.83,AU,['dt']
3,3,palm coast,29.5850,-81.2078,291.00,86,100,1.34,US,['dt']
4,4,adamstown,-25.0660,-130.1015,295.82,91,100,10.34,PN,['dt']


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    xlabel="Longitude", 
    ylabel="Latitude" 
)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_Cities = city_data_df[city_data_df["Max Temp"] > 23]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
Ideal_Cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,252.48,59,20,4.00,RU,['dt']
1,1,mejit,10.2753,170.8646,300.20,80,95,10.34,MH,['dt']
2,2,blackmans bay,-43.0167,147.3167,289.55,60,18,1.83,AU,['dt']
3,3,palm coast,29.5850,-81.2078,291.00,86,100,1.34,US,['dt']
4,4,adamstown,-25.0660,-130.1015,295.82,91,100,10.34,PN,['dt']


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,252.48,59,20,4.00,RU,['dt'],
1,1,mejit,10.2753,170.8646,300.20,80,95,10.34,MH,['dt'],
2,2,blackmans bay,-43.0167,147.3167,289.55,60,18,1.83,AU,['dt'],
3,3,palm coast,29.5850,-81.2078,291.00,86,100,1.34,US,['dt'],
4,4,adamstown,-25.0660,-130.1015,295.82,91,100,10.34,PN,['dt'],


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    longitude = hotel_df.loc[index,"Lng"] 
    latitude = hotel_df.loc[index,"Lat"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
mejit - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
palm coast - nearest hotel: Days Inn
adamstown - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
margaret river - nearest hotel: Margaret River Hotel
thompson - nearest hotel: Thompson Inn
kodiak - nearest hotel: Shelikof Lodge
yellowknife - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
illapel - nearest hotel: Domingo Ortiz de Rosas
manihari - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
saudarkrokur - nearest hotel: Mikligarður
palmerston north - nearest hotel: The Empire Hotel
port-aux-francais - nearest hotel: Keravel
yekaterinoslavka - nearest hotel: No hotel found
sile - nearest hotel: No hotel found
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
santa cruz - nearest hotel: Aviador Hotel Boutique
vorgashor - nearest hotel: Соф

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,252.48,59,20,4.00,RU,['dt'],ООО Постоялый двор
1,1,mejit,10.2753,170.8646,300.20,80,95,10.34,MH,['dt'],No hotel found
2,2,blackmans bay,-43.0167,147.3167,289.55,60,18,1.83,AU,['dt'],Villa Howden
3,3,palm coast,29.5850,-81.2078,291.00,86,100,1.34,US,['dt'],Days Inn
4,4,adamstown,-25.0660,-130.1015,295.82,91,100,10.34,PN,['dt'],No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
552,552,bol,13.4586,14.7147,294.70,39,10,8.18,TD,['dt'],No hotel found
553,553,saint-etienne-de-montluc,47.2769,-1.7807,284.03,89,100,3.09,FR,['dt'],Brit Hotel Atlantel
554,554,barranca,-10.7500,-77.7667,295.04,77,100,3.79,PE,['dt'],Emperador
555,555,aswan,24.0934,32.9070,288.76,38,0,0.00,EG,['dt'],Yaseen hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    xlabel="Longitude", 
    ylabel="Latitude",
    hover_cols=["Hotel Name", "Country"]  
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)